# December 09, 2024

https://adventofcode.com/2024/day/089

In [6]:
from collections import defaultdict

In [144]:
def dprint( *args ):
    if DEBUG:
        return print(*args)


In [1]:
test_str = f'''2333133121414131402'''

In [3]:
fn = "../data/2024/09.txt"
with open(fn, "r") as file:
    text = file.readlines()
puzz_str = [line.strip() for line in text][0]

# Part 1

In [12]:
tot = 0
sum( [int(x) for x in puzz_str] )

94972

In [13]:
len(puzz_str)

19999

In [68]:
def create_disk_map( line ):
    file_blocks = dict()
    empty_blocks = list()

    # toggle between reading a file spec or empty space spec
    is_file = True
    # count file_ids with each new file spec
    file_id = 0
    # keep track of position on the disk
    disk_pos = 0

    for x in line:
        if is_file:
            file_blocks[ file_id ] = [ disk_pos + counter for counter in range(0, int(x)) ]
            file_id += 1
        else:
            empty_blocks += [ disk_pos + counter for counter in range(0, int(x)) ]

        # advance pointer to disk position
        disk_pos += int(x)
        is_file = not is_file

    return file_blocks, empty_blocks


In [69]:
def defrag_disk( file_blocks, empty_space ):
    for file_id in reversed(file_blocks):
        for i, disk_pos in enumerate( file_blocks[file_id][::-1] ):
            #print(file_id, i, disk_pos)

            # only move file block if it goes left
            old_block_pos = file_blocks[file_id][-(i+1)]
            emp = empty_space[0]
            if old_block_pos > emp:
                file_blocks[file_id][-(i+1)] = emp
                empty_space = empty_space[1:] + [old_block_pos]
            # otherwise, exit now
            else:
                return file_blocks, empty_space

In [70]:
def checksum( file_blocks ):
    tot = 0
    for file_id, disk_pos_list in file_blocks.items():
        tot += file_id * sum(disk_pos_list)

    return tot

In [71]:
def part1( line ):
    file_blocks, empty_space = create_disk_map( line )
    file_blocks, empty_space = defrag_disk( file_blocks, empty_space )
    return checksum( file_blocks )


In [72]:
part1(test_str)

1928

In [73]:
part1(puzz_str)

6430446922192

# Part 2

Just doing a completely different method for part 2 today

In [158]:
def create_disk_map2( line ):
    # characterized by start_pos and length
    file_blocks = dict()
    # in retrospect, this could be a list
    empty_blocks = dict()

    # toggle between reading a file spec or empty space spec
    is_file = True
    # count file_ids with each new file spec
    file_id = 0
    empty_id = 0
    # keep track of position on the disk
    disk_pos = 0

    for x in line:
        if is_file:
            file_blocks[ file_id ] = [disk_pos, int(x)]
            file_id += 1
        else:
            empty_blocks[ empty_id ] = [disk_pos, int(x)]
            empty_id += 1

        # advance pointer to disk position
        disk_pos += int(x)
        is_file = not is_file

    return file_blocks, empty_blocks


In [201]:
def clean_empty_space( empty_blocks ):
    # simplify(?) things by removing len-0 empty space
    
    to_del = list()
    for id, spec in empty_blocks.items():
        if spec[1] == 0:
            to_del.append( id )

    if len(to_del) > 0:
        dprint("Removing spaces", ",".join((str(x) for x in to_del)))
        new_empty_dict = dict()
        empty_id = 0
        for id, spec in empty_blocks.items():
            if id not in to_del:
                new_empty_dict[empty_id] = spec
                empty_id += 1

    else:
        new_empty_dict = empty_blocks

    return new_empty_dict

def create_space( empty_blocks, new_spec ):

    new_empty_dict = dict()
    space_added = False
    empty_id = 0

    # now sort them
    for id, spec in empty_blocks.items():
        if not space_added and new_spec[0] < spec[0]:
            new_empty_dict[ empty_id ] = new_spec
            empty_id += 1
            space_added = True
        
        new_empty_dict[ empty_id ] = spec
        empty_id += 1

    return new_empty_dict



def defrag_disk2( file_blocks, empty_blocks ):

    for file_id in reversed( file_blocks ):
        # skipping this resolves an edge case where there's no space immediately before the last file
        # the extra spaces won't harm anything on the first iteration            

        dprint("defrag", file_id)
        cur_pos = file_blocks[file_id][0]
        req_space = file_blocks[file_id][1]

        # after we move the file, we need to keep iterating over the empty space to update it with the recently vacated positions
        file_moved = False

        # This determines if a new space has been created...
        # I'm starting to rethink this whole framework, should I include len-0 spaces going forward?
        # I'm wondering if that will make identifying the appropriate space more difficult.
        new_space = None

        # look for space for this file...
        for empty_id, spec in empty_blocks.items():

            # see if we can move the file
            if not file_moved:
                # don't move stuff to the right - break early
                if spec[0] > cur_pos:
                    break

                # did we find space?
                if spec[1] >= req_space:
                    
                    dprint("moving to emptyblock", empty_id, *spec)
                    # yes! move that file in here!
                    file_blocks[file_id][0] = spec[0]

                    # note: deleting dict entry mid-loop causes runtime error. so just deal with a bunch of length-0 empty spaces
                    # the empty block is now smaller and starts further right
                    empty_blocks[empty_id][0] += req_space
                    empty_blocks[empty_id][1] -= req_space

                    file_moved = True

            # see if we can fix the empty space description
            else: # file_moved is True

                # CASE 1: empty space precedes the file
                if spec[0] + spec[1] == cur_pos:
                    dprint("expanding empty space", empty_id)
                    # This is the empty block right before the file (before it moved)
                    # create a bigger empty space combining this with the now empty file blocks
                    empty_blocks[empty_id][1] += req_space

                    # if the next empty space starts where this empty space ends, we need to combine them
                    if empty_id < len(empty_blocks) - 1 and \
                         empty_blocks[empty_id+1][0] == empty_blocks[empty_id][0] + empty_blocks[empty_id][1]:
                        # add the blocks from second space to the previous one

                        empty_blocks[empty_id][1] += empty_blocks[empty_id+1][1]
                        # remove them from the second space
                        empty_blocks[empty_id+1][1] = 0

                    empty_blocks = clean_empty_space( empty_blocks )
                    # nothing left to update for this move
                    break

                # CASE 2: we create a new empty space
                if spec[0] > cur_pos:

                    # is there an empty space immediately after?
                    if spec[0] == cur_pos + req_space:
                        dprint("expanding following empty space", empty_id)
                        # yes - it starts at cur_pos now and has more blocks
                        empty_blocks[empty_id][0] = cur_pos
                        empty_blocks[empty_id][1] += req_space
                        empty_blocks = clean_empty_space( empty_blocks )

                    else:
                        dprint("creating new empty space")
                        # no - create a new space
                        # I think we can add a new key since we're about to break out of the loop anyway...
                        empty_blocks = create_space( empty_blocks, [cur_pos, req_space] )
                        empty_blocks = clean_empty_space( empty_blocks )

                    break
            
        dprint(file_blocks)
        dprint(empty_blocks)

    dprint("done!")
    dprint(file_blocks)
    dprint(empty_blocks)

    return file_blocks, empty_blocks

def checksum2( f ):
    tot = 0
    for id, spec in f.items():
        # avg * len * file_id
        val = (spec[0] + (spec[1]-1)/2) * spec[1] * id
        dprint(id, val)
        tot += val

    return tot





In [217]:
def part2( puzz ):
    f,e = create_disk_map2( puzz )
    f,e = defrag_disk2( f, e )
    return checksum2( f )
    

In [203]:
DEBUG = True
f, e = create_disk_map2( test_str )
print("--- files ---")
print(f)
print("--- empty ---")
print(e)

f,e = defrag_disk2( f, e )
print("\n--- files after ---")
print(f)
print("--- empty after ---")
print(e)

--- files ---
{0: [0, 2], 1: [5, 3], 2: [11, 1], 3: [15, 3], 4: [19, 2], 5: [22, 4], 6: [27, 4], 7: [32, 3], 8: [36, 4], 9: [40, 2]}
--- empty ---
{0: [2, 3], 1: [8, 3], 2: [12, 3], 3: [18, 1], 4: [21, 1], 5: [26, 1], 6: [31, 1], 7: [35, 1], 8: [40, 0]}
defrag 9
moving to emptyblock 0 2 3
expanding empty space 8
{0: [0, 2], 1: [5, 3], 2: [11, 1], 3: [15, 3], 4: [19, 2], 5: [22, 4], 6: [27, 4], 7: [32, 3], 8: [36, 4], 9: [2, 2]}
{0: [4, 1], 1: [8, 3], 2: [12, 3], 3: [18, 1], 4: [21, 1], 5: [26, 1], 6: [31, 1], 7: [35, 1], 8: [40, 2]}
defrag 8
{0: [0, 2], 1: [5, 3], 2: [11, 1], 3: [15, 3], 4: [19, 2], 5: [22, 4], 6: [27, 4], 7: [32, 3], 8: [36, 4], 9: [2, 2]}
{0: [4, 1], 1: [8, 3], 2: [12, 3], 3: [18, 1], 4: [21, 1], 5: [26, 1], 6: [31, 1], 7: [35, 1], 8: [40, 2]}
defrag 7
moving to emptyblock 1 8 3
expanding empty space 6
Removing spaces 1,7
{0: [0, 2], 1: [5, 3], 2: [11, 1], 3: [15, 3], 4: [19, 2], 5: [22, 4], 6: [27, 4], 7: [8, 3], 8: [36, 4], 9: [2, 2]}
{0: [4, 1], 1: [12, 3], 2: [18

In [207]:
DEBUG = True
print(f)
checksum2( f )

{0: [0, 2], 1: [5, 3], 2: [4, 1], 3: [15, 3], 4: [12, 2], 5: [22, 4], 6: [27, 4], 7: [8, 3], 8: [36, 4], 9: [2, 2]}
0 0.0
1 18.0
2 8.0
3 144.0
4 100.0
5 470.0
6 684.0
7 189.0
8 1200.0
9 45.0


2858.0

In [218]:
DEBUG = False
part2( test_str )

2858.0

In [219]:
part2( puzz_str )

6460170593016.0